# A Simple Hash Table with Linear Probing

In this exercise, your task is to implement a variant of a hash table.  Several simplifications will make this task easier.  First, your table will have a fixed size; it will not be capable of expanding to fit more data.  Your table will only accept strings as keys, though values may be any Python object.  Finally, you will use linear probing to resolve collisions.

Create a class, MyTable, with the following properties:

**Table:**  Your table will have a fixed size, which you should pass in as a parameter to the initializer.  Specifically, you should create a list to store keys (named keys) and a list to store values (named values).  All items in these lists should initially be set to the object None.

**Keys and Values:** The keys to your table will be strings.  Values may be any python object.

**Hashcode:**  Your class should convert each character in a key to its unicode code point (use python’s ord function) and then simply sum them together.

**Compression function:**  To ensure the results of your hashcode falls in the right range, use the mod operator (%) with the size of the hash table.

**Collision resolution:**  You will use linear probing to resolve collisions.  If a particular location in the table is filled, you move forward one space until an empty location is found.  If you reach the end of the table, you cycle back to index 0.

**Deletions:** As with any open addressing system, deletions must be executed with care.  Finding one item A may rely on the fact that item B was in a particular location when A was inserted.  To get around this problem, you should store three types of objects in your list of keys.  The object None indicates that a space has never been used.  The special string “deleted” indicates that the space was used before but is now available.  All other strings represent keys that have been stored in the table.



Inside your MyTable class, you must implement the following methods:

- \_\_setitem\_\_(key, value) - insert the given key-value pair into the table.  If the given key is already in the table, replace the old value with the new value.

- \_\_getitem\_\_(key) - get the value that corresponds to the given key in the table.  

- \_\_delitem\_\_(key) - remove the given key and its corresponding value from the table.  Replace both with the special string “deleted”.

Note that these are magic methods that should not be accessed directly, but will be called when indexing instances of your class with square brackets

In case \_\_getitem\_\_ is called with a key that is not in the table, return the string. “Key not in table.”

Additionally, you should only access your keys list one index at a time and avoid looping through all items in the list whenever possible.  This also means that you should not use operators like *in* that implicitely loop through all items in your list.


The following code demonstrates the proper use of the MyTable class.  Make sure that your class replicates this behavior exactly.



In [249]:
from collections import deque

class MyTable:
    """Table has fixed size. Keys must be string objects. 
    Values can be any Python object. Linear probing used to resolve collisions."""
    
    def __init__(self, size):
        # Initialize key/value lists as fixed sizes
        try:
            size = int(size)
            self.size = size
            self.capacity = size
            self.keys = [None]*size
            self.values = [None]*size
        except:
            print("Size input must be an int object")
    
    def __setitem__(self,key,value):
        try:
            if type(key) != str:
                # Make sure key input is a str object
                raise Exception("Key must be convertable to a string input")

            # Run hash code to figure out index
            index = self.compression(self.hashfunction(key))
            filled = False
            nextindex = None
            
            while filled == False:
                if self.keys[index] == key:
                    # If key is already in the list, then replace the old value
                    self.values[index] = value
                    filled = True
                    
                elif self.keys[index] == None and self.values[index] == None:
                    # If item in index is None, then put key and value into this index
                    self.keys[index] = key
                    self.values[index] = value
                    self.capacity -= 1
                    filled = True
                    
                elif self.keys[index] == "deleted" and self.values[index] == "deleted":
                    # If item in index is "deleted", first put data into that slot
                    self.keys[index] = key
                    self.values[index] = value
                    
                    # We should check other slots to see if that key already exists. Create a separate index for this.
                    if index == self.size - 1:
                        index2 = 0
                    else:
                        index2 = index + 1
                    
                    check = False
                    
                    while check == False:
                        if self.keys[index2] == key:
                            # The key is found to have a repeat, change the previous slot back to "deleted"
                            self.keys[index] = "deleted"
                            self.values[index] = "deleted"
                            # Increment the index and break from the while loop so that the existing key can be updated.
                            index += 1
                            break
                        else:
                            # Implement the loop through this inner while loop to check if the key exists in the table.
                            if index2 == self.size - 1:
                                index3 = 0
                            else:
                                index3 = index2 + 1
                            if index3 == index:
                                # In this case, we did a full loop and the key isn't found in list.
                                # We then break from both loops and replaced the "deleted"
                                self.capacity -= 1
                                filled = True
                                break
                            else:
                                # Continue the loop to see if the key exists in the table already.
                                index2 = index3
                            
                else:
                    # Otherwise, go to the next index
                    if index == self.size - 1:
                        nextindex = 0
                    else:
                        nextindex = index + 1
                    if nextindex == self.compression(self.hashfunction(key)):
                        # If we've made a complete loop, then break and print hash table is full
                        print("Hash Table is Full")
                        break                     
                    else:
                        index = nextindex
                         
        except Exception as e:
            # Print error if key value isn't a str object
            print(e)
            
    def __getitem__(self,key):
        try:
            if type(key) != str:
                # Make sure key input is a str object
                raise Exception("Key must be convertable to a string input")
                
            # Run hash code to figure out index 
            index = self.compression(self.hashfunction(key))
            retrieved = False
            nextindex = None
                
            while retrieved == False:
                if self.keys[index] == key:
                    # If the key stored in the index is what you want, return the value for that index
                    retrieved = True
                    return self.values[index]
                elif self.keys[index] == None:
                    # If keys are not in hash table, let the user know
                    retrieved = True
                    return "Key not in table."
                else:
                    # Otherwise, go to the next index
                    if index == self.size - 1:
                        nextindex = 0
                    else:
                        nextindex = index + 1
                    if nextindex == self.compression(self.hashfunction(key)):
                        # If we've made a complete loop, then break and print hash table is full
                        print("Key not in table.")
                        break                     
                    else:
                        index = nextindex
                
        except Exception as e:
            # Print error if key value isn't a str object
            print(e)
            
    def __delitem__(self,key):
        try:
            if type(key) != str:
                # Make sure key input is a str object
                raise Exception("Key must be convertable to a string input")
            
            # Run hash code to figure out index 
            index = self.compression(self.hashfunction(key))
            deleted = False
            nextindex = None
            
            while deleted == False:
                if self.keys[index] == key:
                    # If the key stored in the index is what you want, deleted the key/value pair
                    self.keys[index] = "deleted"
                    self.values[index] = "deleted"
                    deleted = True
                elif self.keys[index] == None:
                    # If keys are not in the hash table, let the user know
                    print("Key not in table.")
                    break
                else:
                    # Otherwise, check the next index
                    if index == self.size - 1:
                        nextindex = 0
                    else:
                        nextindex = index + 1
                    if nextindex == self.compression(self.hashfunction(key)):
                        # If we've made a complete loop, then break and print hash table is full
                        print("Key not in table.")
                        break
                    else:
                        index = nextindex
                
        except Exception as e:
            # Print error if key value isn't a str object
            print(e)
        
    def hashfunction(self,key):
        # Convert each character in a key to its unicode code point and then sum them together
        return sum(ord(char) for char in key)
    
    def compression(self,hashvalue):
        # Use the mod operator with the size of the hash table
        return hashvalue%len(self.values)
    

In [324]:
m = MyTable(5)
# The following keys all hash to the same index.
m["a"] = "apple"
m["f"] = "butter" 
m["k"] = "yummy"
print("Current keys in m:", m.keys)

Current keys in m: [None, None, 'a', 'f', 'k']


In [325]:
# "p" also hashes to the same place.
# Your class should detect that it's not in the table
# without scanning through the entire keys list.
print("m['p']:", m["p"])

m['p']: Key not in table.


In [326]:
# We can access key "k"
print("m['k']:", m["k"])
# Even if we remove "f"
del m["f"]
print("m['k']:", m["k"])
print("Current keys in m:", m.keys)

m['k']: yummy
m['k']: yummy
Current keys in m: [None, None, 'a', 'deleted', 'k']


In [327]:
# Even after removing "f", we can overwrite "k"
m["k"] = "newstuff"
print("Current keys in m:", m.keys)
print("Current values in m:", m.values)

Current keys in m: [None, None, 'a', 'deleted', 'k']
Current values in m: [None, None, 'apple', 'deleted', 'newstuff']
